In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import sklearn
import pandas as pd
import os
import sys
import time
import tensorflow as tf

from tensorflow import keras

print(tf.__version__)

In [ ]:
from sklearn.datasets import fetch_california_housing
housing  = fetch_california_housing()
print(housing.DESCR)
print(housing.data.shape)
print(housing.target.shape)

In [ ]:
import pprint
pprint.pprint(housing.data[0:5])
pprint.pprint(housing.target[0:5])

In [ ]:
from sklearn.model_selection import train_test_split
x_train_all,x_test,y_train_all,y_test = train_test_split(
    housing.data,housing.target,random_state = 7
)
x_train,x_valid,y_train,y_valid = train_test_split(
    x_train_all,y_train_all,random_state = 11
)
print(x_train.shape,y_train.shape)
print(x_test.shape,y_test.shape)
print(x_valid.shape,y_valid.shape)

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)
x_valid_scaled = scaler.transform(x_valid)

In [ ]:
# tf.nn.softplus :log(1+e^x)
customized_softplus = keras.layers.Lambda(lambda x : tf.nn.softplus(x))
print(customized_softplus([-10.,-5.,0.,5.,10.]))

In [ ]:
#customized dense layer
class CustomizedDenseLayer(keras.layers.Layer):
    def __init__(self,units,activation=None,**kwargs):
        self.units = units
        self.activation = keras.layers.Activation(activation)
        super(CustomizedDenseLayer,self).__init__(**kwargs)

    def build(self,input_shape):
        """构建所需要的参数"""
        #x * w + b.  input_shape:[None,a] * w_shape:w_shape = output_shape:[None,b] 
        self.kernel = self.add_weight(name='kernel',shape=(input_shape[1],self.units),initializer='uniform',trainable=True)
        self.bias = self.add_weight(name='bias',shape=(self.units),initializer='zeros',trainable=True)
        super(CustomizedDenseLayer,self).build(input_shape)

    def call(self,x):
        """完成正向计算"""
        return self.activation(x @ self.kernel + self.bias)

In [ ]:
model = keras.models.Sequential([
    CustomizedDenseLayer(30,activation='relu',input_shape=x_train.shape[1:]),
    CustomizedDenseLayer(1),
    customized_softplus,
    #keras.layers.Dense(1,activation="softplus"),
    #keras.layers.Dense(1),keras.layers.Activation('softplus')
])
model.summary()
model.compile(loss="mean_squared_error",optimizer="adam")
callbacks = [keras.callbacks.EarlyStopping(patience=5,min_delta=1e-3)]

In [ ]:
history = model.fit(x_train_scaled,y_train,validation_data=(x_valid_scaled,y_valid),epochs=100,callbacks=callbacks)

In [ ]:
def plot_learning_curves(history):
    pd.DataFrame(history.history).plot(figsize=(8,5))
    plt.grid(True)
    plt.gca().set_ylim(0,1)
    plt.show()
plot_learning_curves(history)

In [ ]:
model.evaluate(x_test_scaled,y_test)